In [1]:
#data base connection
import csv
import pandas
from sshtunnel import SSHTunnelForwarder
import pymysql
from tabulate import tabulate

#specifiy the path for your file
with open('/Users/QN/Desktop/THIRD crowdperm/user.txt') as f:
    lines = f.read().splitlines()    
f.close()


server =    SSHTunnelForwarder(
         ('juhu.soic.indiana.edu'),
         ssh_username=lines[0],
         ssh_password=lines[1],
         remote_bind_address=('127.0.0.1', 3306))
server.start()
cnx = pymysql.connect(host='127.0.0.1',
                           port=server.local_bind_port,
                           user='mturk',
                           passwd='m+turk=my-sql',
                           db='mturk')
cnx.set_charset('utf8') 
cursor = cnx.cursor()

query = ("select * from survey2")
cursor.execute(query)

allowAll={"I want to use a specific feature that requires this permission":0,
          "I think the app wont work otherwise":0,
          "I trust the app developer":0,
          "Because the app is popular":0,
          "I wont be able to grant this permission later":0,
          "I have nothing to hide":0,
          "I wanted the permission screen to go away":0,
          "Nothing bad will happen":0,
          "I didnt know I did that":0,
          "I dont know":0,
          "The app developer already has this information about me":0,
          "Other":0}
disAll={"I do not use the specific feature associated with the permission":0,
        "I think the app shouldnt need this permission":0,
        "I expect the app will still work without this permission":0,
        "I consider the permission to be very sensitive":0,
        "I dont trust the developer enough to provide this information":0,
        "I can always grant it afterwards if I change my mind":0,
        "I wanted the permission screen to go away":0,
        "I think something bad might happen if I provide this permission":0,
        "I didnt know I did that":0,
        "I dont know":0,
        "I dont use the app":0,
        "The app did not ask for that permission/I did not disable the permission/I think thats the default setting":0,
        "Other":0}

for row in cursor:
    reasonY = row[18].split(',')
    reasonN = row[20].split(',')
    for x in reasonY:
        x = x.lstrip()
        x = x.rstrip()
        x=x.replace('\'','')
        x=x.replace('\\','')
        if x[0:5] == "Other": allowAll["Other"] += 1
        elif x=="NA": pass
        elif x == "I want to use a specific feature that requires this permission": allowAll[x] += 1
        elif x == "I think the app wont work otherwise": allowAll[x] += 1
        elif x == "I trust the app developer": allowAll[x] += 1
        elif x == "Because the app is popular": allowAll[x] += 1
        elif x=="I wont be able to grant this permission later":allowAll[x] += 1
        elif x== "I have nothing to hide":allowAll[x] += 1
        elif x== "I wanted the permission screen to go away":allowAll[x] += 1
        elif x== "Nothing bad will happen":allowAll[x] += 1
        elif x== "I didnt know I did that":allowAll[x] += 1
        elif x== "I dont know":allowAll[x] += 1
        elif x== "The app developer already has this information about me":allowAll[x] += 1        
        
    
    for x in reasonN:
             
        x = x.lstrip()
        x = x.rstrip()
        x=x.replace('\'','')
        x=x.replace('\\','')
        x=x.replace("thatâ€™s","thats")
        x=x.replace("donâ€™t","dont")
        if x[0:5] == "Other": disAll["Other"] += 1
        elif x=="NA": pass
        elif x=="I do not use the specific feature associated with the permission": disAll[x] += 1
        elif x=="I think the app shouldnt need this permission": disAll[x] += 1
        elif x=="I expect the app will still work without this permission": disAll[x] += 1
        elif x=="I consider the permission to be very sensitive": disAll[x] += 1
        elif x=="I dont trust the developer enough to provide this information": disAll[x] += 1
        elif x=="I can always grant it afterwards if I change my mind": disAll[x] += 1
        elif x=="I wanted the permission screen to go away": disAll[x]+= 1
        elif x=="I think something bad might happen if I provide this permission": disAll[x] += 1
        elif x=="I didnt know I did that": disAll[x] += 1
        elif x=="I dont know": disAll[x] += 1
        elif x=="I dont use the app": disAll[x] += 1
        elif x=="The app did not ask for that permission/I did not disable the permission/I think thats the default setting": disAll[x] += 1

allAllow = []
for key in allowAll:
    temp = []
    temp.append(key)
    temp.append(allowAll[key])
    allAllow.append(temp)
allDis = []
for key in disAll:
    temp = []
    #make this spesific key shorter for display purposes 
    if key== "The app did not ask for that permission/I did not disable the permission/I think thats the default setting":
        temp.append("The app didn't ask for it/I didn't disable it/I think thats the default setting"+'')    
    else:
         temp.append(key+'')  
    temp.append(disAll[key])
    allDis.append(temp)

allow = pandas.DataFrame(allAllow,columns=['Reason','Count'])
disallow = pandas.DataFrame(allDis,columns=['Reason','Count'])



The most chosen reasons for allowing All permissions:
- I want to use a specific feature that requires this permission
- I trust the app developer  
- I think the app wont work otherwise
- I have nothing to hide 

In [2]:
print(tabulate(allow, showindex=False, headers=allow.columns,tablefmt='psql'))

+----------------------------------------------------------------+---------+
| Reason                                                         |   Count |
|----------------------------------------------------------------+---------|
| I want to use a specific feature that requires this permission |     104 |
| I think the app wont work otherwise                            |      72 |
| I trust the app developer                                      |      96 |
| Because the app is popular                                     |      23 |
| I wont be able to grant this permission later                  |       0 |
| I have nothing to hide                                         |      54 |
| I wanted the permission screen to go away                      |      33 |
| Nothing bad will happen                                        |      32 |
| I didnt know I did that                                        |      32 |
| I dont know                                                    |      20 |

The most chosen reasons for denying all permissions:
- I dont use the app 
- I think the app shouldnt need this permission
- I can always grant it afterwards if I change my mind
- The app didn't ask for it/I didn't disable it/I think thats the default setting 

In [3]:
print(tabulate(disallow, showindex=False, headers=disallow.columns,tablefmt='psql'))

+---------------------------------------------------------------------------------+---------+
| Reason                                                                          |   Count |
|---------------------------------------------------------------------------------+---------|
| I do not use the specific feature associated with the permission                |      30 |
| I think the app shouldnt need this permission                                   |      47 |
| I expect the app will still work without this permission                        |      39 |
| I consider the permission to be very sensitive                                  |      19 |
| I dont trust the developer enough to provide this information                   |      18 |
| I can always grant it afterwards if I change my mind                            |      45 |
| I wanted the permission screen to go away                                       |       4 |
| I think something bad might happen if I provide this permi

The most chosen reasons for allowing Location:
- I want to use a specific feature that requires this permission
- I trust the app developer
- I think the app wont work otherwise
- I didnt know I did that



The most chosen reasons for denying Location:
- I think the app shouldnt need this permission
- I expect the app will still work without this permission 
- I do not use the specific feature associated with the permission 
- I can always grant it afterwards if I change my mind
	


In [4]:
##### for micophone
query = ("select * from survey2 where permission='LOCATION'")
cursor.execute(query)

allowAll={"I want to use a specific feature that requires this permission":0,
          "I think the app wont work otherwise":0,
          "I trust the app developer":0,
          "Because the app is popular":0,
          "I wont be able to grant this permission later":0,
          "I have nothing to hide":0,
          "I wanted the permission screen to go away":0,
          "Nothing bad will happen":0,
          "I didnt know I did that":0,
          "I dont know":0,
          "The app developer already has this information about me":0,
          "Other":0}
disAll={"I do not use the specific feature associated with the permission":0,
        "I think the app shouldnt need this permission":0,
        "I expect the app will still work without this permission":0,
        "I consider the permission to be very sensitive":0,
        "I dont trust the developer enough to provide this information":0,
        "I can always grant it afterwards if I change my mind":0,
        "I wanted the permission screen to go away":0,
        "I think something bad might happen if I provide this permission":0,
        "I didnt know I did that":0,
        "I dont know":0,
        "I dont use the app":0,
        "The app did not ask for that permission/I did not disable the permission/I think thats the default setting":0,
        "Other":0}

for row in cursor:
    reasonY=[]
    reasonN=[]
    reasonY1 = row[42].split(',')
    reasonY.append(reasonY1)
    reasonY2 = row[43].split(',')
    reasonY.append(reasonY2)
    reasonY3 = row[44].split(',')
    reasonY.append(reasonY3)
    reasonY4 = row[45].split(',')
    reasonY.append(reasonY4)
    reasonY5 = row[46].split(',')
    reasonY.append(reasonY5)
    
    reasonN1 = row[53].split(',')
    reasonN.append(reasonN1)
    reasonN2 = row[55].split(',')
    reasonN.append(reasonN2)
    reasonN3 = row[57].split(',')
    reasonN.append(reasonN3)
    reasonN4 = row[59].split(',')
    reasonN.append(reasonN4)
    reasonN5 = row[61].split(',')
    reasonN.append(reasonN5)
    
    for x1 in reasonY:
        for x in x1:
            x = x.lstrip()
            x = x.rstrip()
            x=x.replace('\'','')
            x=x.replace('\\','')
            if x[0:5] == "Other": allowAll["Other"] += 1
            elif x=="NA": pass
            elif x == "I want to use a specific feature that requires this permission": allowAll[x] += 1
            elif x == "I think the app wont work otherwise": allowAll[x] += 1
            elif x == "I trust the app developer": allowAll[x] += 1
            elif x == "Because the app is popular": allowAll[x] += 1
            elif x=="I wont be able to grant this permission later":allowAll[x] += 1
            elif x== "I have nothing to hide":allowAll[x] += 1
            elif x== "I wanted the permission screen to go away":allowAll[x] += 1
            elif x== "Nothing bad will happen":allowAll[x] += 1
            elif x== "I didnt know I did that":allowAll[x] += 1
            elif x== "I dont know":allowAll[x] += 1
            elif x== "The app developer already has this information about me":allowAll[x] += 1        
        
    for x1 in reasonN:
        for x in x1:
             
            x = x.lstrip()
            x = x.rstrip()
            x=x.replace('\'','')
            x=x.replace('\\','')
            x=x.replace("thatâ€™s","thats")
            x=x.replace("donâ€™t","dont")
            if x[0:5] == "Other": disAll["Other"] += 1
            elif x=="NA": pass
            elif x=="I do not use the specific feature associated with the permission": disAll[x] += 1
            elif x=="I think the app shouldnt need this permission": disAll[x] += 1
            elif x=="I expect the app will still work without this permission": disAll[x] += 1
            elif x=="I consider the permission to be very sensitive": disAll[x] += 1
            elif x=="I dont trust the developer enough to provide this information": disAll[x] += 1
            elif x=="I can always grant it afterwards if I change my mind": disAll[x] += 1
            elif x=="I wanted the permission screen to go away": disAll[x]+= 1
            elif x=="I think something bad might happen if I provide this permission": disAll[x] += 1
            elif x=="I didnt know I did that": disAll[x] += 1
            elif x=="I dont know": disAll[x] += 1
            elif x=="I dont use the app": disAll[x] += 1
            elif x=="The app did not ask for that permission/I did not disable the permission/I think thats the default setting": disAll[x] += 1

allAllow = []
for key in allowAll:
    temp = []
    temp.append(key)
    temp.append(allowAll[key])
    allAllow.append(temp)
allDis = []
for key in disAll:
    temp = []
    #make this spesific key shorter for display purposes 
    if key== "The app did not ask for that permission/I did not disable the permission/I think thats the default setting":
        temp.append("The app didn't ask for it/I didn't disable it/I think thats the default setting"+'')    
    else:
         temp.append(key+'')  
    temp.append(disAll[key])
    allDis.append(temp)

allowM = pandas.DataFrame(allAllow,columns=['Reason:LOCATION:Allow','Count'])
disallowM = pandas.DataFrame(allDis,columns=['Reason:LOCATION:Deny','Count'])
print(tabulate(allowM, showindex=False, headers=allowM.columns,tablefmt='psql'))
print(tabulate(disallowM, showindex=False, headers=disallowM.columns,tablefmt='psql'))
#pandas.set_option('max_colwidth',200)

+----------------------------------------------------------------+---------+
| Reason:LOCATION:Allow                                          |   Count |
|----------------------------------------------------------------+---------|
| I want to use a specific feature that requires this permission |     111 |
| I think the app wont work otherwise                            |      33 |
| I trust the app developer                                      |      39 |
| Because the app is popular                                     |      14 |
| I wont be able to grant this permission later                  |       1 |
| I have nothing to hide                                         |      20 |
| I wanted the permission screen to go away                      |       6 |
| Nothing bad will happen                                        |      15 |
| I didnt know I did that                                        |      32 |
| I dont know                                                    |      20 |

The most chosen reasons for allowing Microphone:
- I want to use a specific feature that requires this permission
- I trust the app developer
- I think the app wont work otherwise
- I didnt know I did that



The most chosen reasons for denying Microphone:
- I can always grant it afterwards if I change my mind
- I expect the app will still work without this permission
- I think the app shouldnt need this permission 
- I do not use the specific feature associated with the permission 

In [5]:
##### for micophone

query = ("select * from survey2 where permission='MICROPHONE'")
cursor.execute(query)

allowAll={"I want to use a specific feature that requires this permission":0,
          "I think the app wont work otherwise":0,
          "I trust the app developer":0,
          "Because the app is popular":0,
          "I wont be able to grant this permission later":0,
          "I have nothing to hide":0,
          "I wanted the permission screen to go away":0,
          "Nothing bad will happen":0,
          "I didnt know I did that":0,
          "I dont know":0,
          "The app developer already has this information about me":0,
          "Other":0}
disAll={"I do not use the specific feature associated with the permission":0,
        "I think the app shouldnt need this permission":0,
        "I expect the app will still work without this permission":0,
        "I consider the permission to be very sensitive":0,
        "I dont trust the developer enough to provide this information":0,
        "I can always grant it afterwards if I change my mind":0,
        "I wanted the permission screen to go away":0,
        "I think something bad might happen if I provide this permission":0,
        "I didnt know I did that":0,
        "I dont know":0,
        "I dont use the app":0,
        "The app did not ask for that permission/I did not disable the permission/I think thats the default setting":0,
        "Other":0}

for row in cursor:
    reasonY=[]
    reasonN=[]
    reasonY1 = row[42].split(',')
    reasonY.append(reasonY1)
    reasonY2 = row[43].split(',')
    reasonY.append(reasonY2)
    reasonY3 = row[44].split(',')
    reasonY.append(reasonY3)
    reasonY4 = row[45].split(',')
    reasonY.append(reasonY4)
    reasonY5 = row[46].split(',')
    reasonY.append(reasonY5)
    
    reasonN1 = row[53].split(',')
    reasonN.append(reasonN1)
    reasonN2 = row[55].split(',')
    reasonN.append(reasonN2)
    reasonN3 = row[57].split(',')
    reasonN.append(reasonN3)
    reasonN4 = row[59].split(',')
    reasonN.append(reasonN4)
    reasonN5 = row[61].split(',')
    reasonN.append(reasonN5)
    
    for x1 in reasonY:
        for x in x1:
            x = x.lstrip()
            x = x.rstrip()
            x=x.replace('\'','')
            x=x.replace('\\','')
            if x[0:5] == "Other": allowAll["Other"] += 1
            elif x=="NA": pass
            elif x == "I want to use a specific feature that requires this permission": allowAll[x] += 1
            elif x == "I think the app wont work otherwise": allowAll[x] += 1
            elif x == "I trust the app developer": allowAll[x] += 1
            elif x == "Because the app is popular": allowAll[x] += 1
            elif x=="I wont be able to grant this permission later":allowAll[x] += 1
            elif x== "I have nothing to hide":allowAll[x] += 1
            elif x== "I wanted the permission screen to go away":allowAll[x] += 1
            elif x== "Nothing bad will happen":allowAll[x] += 1
            elif x== "I didnt know I did that":allowAll[x] += 1
            elif x== "I dont know":allowAll[x] += 1
            elif x== "The app developer already has this information about me":allowAll[x] += 1        
        
    for x1 in reasonN:
        for x in x1:
             
            x = x.lstrip()
            x = x.rstrip()
            x=x.replace('\'','')
            x=x.replace('\\','')
            x=x.replace("thatâ€™s","thats")
            x=x.replace("donâ€™t","dont")
            if x[0:5] == "Other": disAll["Other"] += 1
            elif x=="NA": pass
            elif x=="I do not use the specific feature associated with the permission": disAll[x] += 1
            elif x=="I think the app shouldnt need this permission": disAll[x] += 1
            elif x=="I expect the app will still work without this permission": disAll[x] += 1
            elif x=="I consider the permission to be very sensitive": disAll[x] += 1
            elif x=="I dont trust the developer enough to provide this information": disAll[x] += 1
            elif x=="I can always grant it afterwards if I change my mind": disAll[x] += 1
            elif x=="I wanted the permission screen to go away": disAll[x]+= 1
            elif x=="I think something bad might happen if I provide this permission": disAll[x] += 1
            elif x=="I didnt know I did that": disAll[x] += 1
            elif x=="I dont know": disAll[x] += 1
            elif x=="I dont use the app": disAll[x] += 1
            elif x=="The app did not ask for that permission/I did not disable the permission/I think thats the default setting": disAll[x] += 1

allAllow = []
for key in allowAll:
    temp = []
    temp.append(key)
    temp.append(allowAll[key])
    allAllow.append(temp)
allDis = []
for key in disAll:
    temp = []
    #make this spesific key shorter for display purposes 
    if key== "The app did not ask for that permission/I did not disable the permission/I think thats the default setting":
        temp.append("The app didn't ask for it/I didn't disable it/I think thats the default setting"+'')    
    else:
         temp.append(key+'')  
    temp.append(disAll[key])
    allDis.append(temp)

allowM = pandas.DataFrame(allAllow,columns=['Reason:MICROPHONE:Allow','Count'])
disallowM = pandas.DataFrame(allDis,columns=['Reason:MICROPHONE:Deny','Count'])
print(tabulate(allowM, showindex=False, headers=allowM.columns,tablefmt='psql'))
print(tabulate(disallowM, showindex=False, headers=disallowM.columns,tablefmt='psql'))
#pandas.set_option('max_colwidth',200)

+----------------------------------------------------------------+---------+
| Reason:MICROPHONE:Allow                                        |   Count |
|----------------------------------------------------------------+---------|
| I want to use a specific feature that requires this permission |      54 |
| I think the app wont work otherwise                            |      18 |
| I trust the app developer                                      |      29 |
| Because the app is popular                                     |       5 |
| I wont be able to grant this permission later                  |       2 |
| I have nothing to hide                                         |      19 |
| I wanted the permission screen to go away                      |       8 |
| Nothing bad will happen                                        |       6 |
| I didnt know I did that                                        |      24 |
| I dont know                                                    |       8 |

In [6]:
##### for camera
query = ("select * from survey2 where permission='CAMERA'")
cursor.execute(query)

allowAll={"I want to use a specific feature that requires this permission":0,
          "I think the app wont work otherwise":0,
          "I trust the app developer":0,
          "Because the app is popular":0,
          "I wont be able to grant this permission later":0,
          "I have nothing to hide":0,
          "I wanted the permission screen to go away":0,
          "Nothing bad will happen":0,
          "I didnt know I did that":0,
          "I dont know":0,
          "The app developer already has this information about me":0,
          "Other":0}
disAll={"I do not use the specific feature associated with the permission":0,
        "I think the app shouldnt need this permission":0,
        "I expect the app will still work without this permission":0,
        "I consider the permission to be very sensitive":0,
        "I dont trust the developer enough to provide this information":0,
        "I can always grant it afterwards if I change my mind":0,
        "I wanted the permission screen to go away":0,
        "I think something bad might happen if I provide this permission":0,
        "I didnt know I did that":0,
        "I dont know":0,
        "I dont use the app":0,
        "The app did not ask for that permission/I did not disable the permission/I think thats the default setting":0,
        "Other":0}

for row in cursor:
    reasonY=[]
    reasonN=[]
    reasonY1 = row[42].split(',')
    reasonY.append(reasonY1)
    reasonY2 = row[43].split(',')
    reasonY.append(reasonY2)
    reasonY3 = row[44].split(',')
    reasonY.append(reasonY3)
    reasonY4 = row[45].split(',')
    reasonY.append(reasonY4)
    reasonY5 = row[46].split(',')
    reasonY.append(reasonY5)
    
    reasonN1 = row[53].split(',')
    reasonN.append(reasonN1)
    reasonN2 = row[55].split(',')
    reasonN.append(reasonN2)
    reasonN3 = row[57].split(',')
    reasonN.append(reasonN3)
    reasonN4 = row[59].split(',')
    reasonN.append(reasonN4)
    reasonN5 = row[61].split(',')
    reasonN.append(reasonN5)
    
    for x1 in reasonY:
        for x in x1:
            x = x.lstrip()
            x = x.rstrip()
            x=x.replace('\'','')
            x=x.replace('\\','')
            if x[0:5] == "Other": allowAll["Other"] += 1
            elif x=="NA": pass
            elif x == "I want to use a specific feature that requires this permission": allowAll[x] += 1
            elif x == "I think the app wont work otherwise": allowAll[x] += 1
            elif x == "I trust the app developer": allowAll[x] += 1
            elif x == "Because the app is popular": allowAll[x] += 1
            elif x=="I wont be able to grant this permission later":allowAll[x] += 1
            elif x== "I have nothing to hide":allowAll[x] += 1
            elif x== "I wanted the permission screen to go away":allowAll[x] += 1
            elif x== "Nothing bad will happen":allowAll[x] += 1
            elif x== "I didnt know I did that":allowAll[x] += 1
            elif x== "I dont know":allowAll[x] += 1
            elif x== "The app developer already has this information about me":allowAll[x] += 1        
        
    for x1 in reasonN:
        for x in x1:
             
            x = x.lstrip()
            x = x.rstrip()
            x=x.replace('\'','')
            x=x.replace('\\','')
            x=x.replace("thatâ€™s","thats")
            x=x.replace("donâ€™t","dont")
            if x[0:5] == "Other": disAll["Other"] += 1
            elif x=="NA": pass
            elif x=="I do not use the specific feature associated with the permission": disAll[x] += 1
            elif x=="I think the app shouldnt need this permission": disAll[x] += 1
            elif x=="I expect the app will still work without this permission": disAll[x] += 1
            elif x=="I consider the permission to be very sensitive": disAll[x] += 1
            elif x=="I dont trust the developer enough to provide this information": disAll[x] += 1
            elif x=="I can always grant it afterwards if I change my mind": disAll[x] += 1
            elif x=="I wanted the permission screen to go away": disAll[x]+= 1
            elif x=="I think something bad might happen if I provide this permission": disAll[x] += 1
            elif x=="I didnt know I did that": disAll[x] += 1
            elif x=="I dont know": disAll[x] += 1
            elif x=="I dont use the app": disAll[x] += 1
            elif x=="The app did not ask for that permission/I did not disable the permission/I think thats the default setting": disAll[x] += 1

allAllow = []
for key in allowAll:
    temp = []
    temp.append(key)
    temp.append(allowAll[key])
    allAllow.append(temp)
allDis = []
for key in disAll:
    temp = []
    #make this spesific key shorter for display purposes 
    if key== "The app did not ask for that permission/I did not disable the permission/I think thats the default setting":
        temp.append("The app didn't ask for it/I didn't disable it/I think thats the default setting"+'')    
    else:
         temp.append(key+'')  
    temp.append(disAll[key])
    allDis.append(temp)


allowM = pandas.DataFrame(allAllow,columns=['Reason:CAMERA:Allow','Count'])
disallowM = pandas.DataFrame(allDis,columns=['Reason:CAMERA:Deny','Count'])
print(tabulate(allowM, showindex=False, headers=allowM.columns,tablefmt='psql'))
print(tabulate(disallowM, showindex=False, headers=disallowM.columns,tablefmt='psql'))
#pandas.set_option('max_colwidth',200)

+----------------------------------------------------------------+---------+
| Reason:CAMERA:Allow                                            |   Count |
|----------------------------------------------------------------+---------|
| I want to use a specific feature that requires this permission |      83 |
| I think the app wont work otherwise                            |      13 |
| I trust the app developer                                      |      31 |
| Because the app is popular                                     |       7 |
| I wont be able to grant this permission later                  |       2 |
| I have nothing to hide                                         |      15 |
| I wanted the permission screen to go away                      |       7 |
| Nothing bad will happen                                        |       8 |
| I didnt know I did that                                        |      20 |
| I dont know                                                    |      10 |

In [7]:
##### for contacts
query = ("select * from survey2 where permission='CONTACTS'")
cursor.execute(query)

allowAll={"I want to use a specific feature that requires this permission":0,
          "I think the app wont work otherwise":0,
          "I trust the app developer":0,
          "Because the app is popular":0,
          "I wont be able to grant this permission later":0,
          "I have nothing to hide":0,
          "I wanted the permission screen to go away":0,
          "Nothing bad will happen":0,
          "I didnt know I did that":0,
          "I dont know":0,
          "The app developer already has this information about me":0,
          "Other":0}
disAll={"I do not use the specific feature associated with the permission":0,
        "I think the app shouldnt need this permission":0,
        "I expect the app will still work without this permission":0,
        "I consider the permission to be very sensitive":0,
        "I dont trust the developer enough to provide this information":0,
        "I can always grant it afterwards if I change my mind":0,
        "I wanted the permission screen to go away":0,
        "I think something bad might happen if I provide this permission":0,
        "I didnt know I did that":0,
        "I dont know":0,
        "I dont use the app":0,
        "The app did not ask for that permission/I did not disable the permission/I think thats the default setting":0,
        "Other":0}

for row in cursor:
    reasonY=[]
    reasonN=[]
    reasonY1 = row[42].split(',')
    reasonY.append(reasonY1)
    reasonY2 = row[43].split(',')
    reasonY.append(reasonY2)
    reasonY3 = row[44].split(',')
    reasonY.append(reasonY3)
    reasonY4 = row[45].split(',')
    reasonY.append(reasonY4)
    reasonY5 = row[46].split(',')
    reasonY.append(reasonY5)
    
    reasonN1 = row[53].split(',')
    reasonN.append(reasonN1)
    reasonN2 = row[55].split(',')
    reasonN.append(reasonN2)
    reasonN3 = row[57].split(',')
    reasonN.append(reasonN3)
    reasonN4 = row[59].split(',')
    reasonN.append(reasonN4)
    reasonN5 = row[61].split(',')
    reasonN.append(reasonN5)
    
    for x1 in reasonY:
        for x in x1:
            x = x.lstrip()
            x = x.rstrip()
            x=x.replace('\'','')
            x=x.replace('\\','')
            if x[0:5] == "Other": allowAll["Other"] += 1
            elif x=="NA": pass
            elif x == "I want to use a specific feature that requires this permission": allowAll[x] += 1
            elif x == "I think the app wont work otherwise": allowAll[x] += 1
            elif x == "I trust the app developer": allowAll[x] += 1
            elif x == "Because the app is popular": allowAll[x] += 1
            elif x=="I wont be able to grant this permission later":allowAll[x] += 1
            elif x== "I have nothing to hide":allowAll[x] += 1
            elif x== "I wanted the permission screen to go away":allowAll[x] += 1
            elif x== "Nothing bad will happen":allowAll[x] += 1
            elif x== "I didnt know I did that":allowAll[x] += 1
            elif x== "I dont know":allowAll[x] += 1
            elif x== "The app developer already has this information about me":allowAll[x] += 1        
        
    for x1 in reasonN:
        for x in x1:
             
            x = x.lstrip()
            x = x.rstrip()
            x=x.replace('\'','')
            x=x.replace('\\','')
            x=x.replace("thatâ€™s","thats")
            x=x.replace("donâ€™t","dont")
            if x[0:5] == "Other": disAll["Other"] += 1
            elif x=="NA": pass
            elif x=="I do not use the specific feature associated with the permission": disAll[x] += 1
            elif x=="I think the app shouldnt need this permission": disAll[x] += 1
            elif x=="I expect the app will still work without this permission": disAll[x] += 1
            elif x=="I consider the permission to be very sensitive": disAll[x] += 1
            elif x=="I dont trust the developer enough to provide this information": disAll[x] += 1
            elif x=="I can always grant it afterwards if I change my mind": disAll[x] += 1
            elif x=="I wanted the permission screen to go away": disAll[x]+= 1
            elif x=="I think something bad might happen if I provide this permission": disAll[x] += 1
            elif x=="I didnt know I did that": disAll[x] += 1
            elif x=="I dont know": disAll[x] += 1
            elif x=="I dont use the app": disAll[x] += 1
            elif x=="The app did not ask for that permission/I did not disable the permission/I think thats the default setting": disAll[x] += 1

allAllow = []
for key in allowAll:
    temp = []
    temp.append(key)
    temp.append(allowAll[key])
    allAllow.append(temp)
allDis = []
for key in disAll:
    temp = []
    #make this spesific key shorter for display purposes 
    if key== "The app did not ask for that permission/I did not disable the permission/I think thats the default setting":
        temp.append("The app didn't ask for it/I didn't disable it/I think thats the default setting"+'')    
    else:
         temp.append(key+'')  
    temp.append(disAll[key])
    allDis.append(temp)



allowM = pandas.DataFrame(allAllow,columns=['Reason:CONTACTS:Allow','Count'])
disallowM = pandas.DataFrame(allDis,columns=['Reason:CONTACTS:Deny','Count'])
print(tabulate(allowM, showindex=False, headers=allowM.columns,tablefmt='psql'))
print(tabulate(disallowM, showindex=False, headers=disallowM.columns,tablefmt='psql'))
#pandas.set_option('max_colwidth',200)

+----------------------------------------------------------------+---------+
| Reason:CONTACTS:Allow                                          |   Count |
|----------------------------------------------------------------+---------|
| I want to use a specific feature that requires this permission |      48 |
| I think the app wont work otherwise                            |      17 |
| I trust the app developer                                      |      43 |
| Because the app is popular                                     |      14 |
| I wont be able to grant this permission later                  |       0 |
| I have nothing to hide                                         |      25 |
| I wanted the permission screen to go away                      |       5 |
| Nothing bad will happen                                        |      23 |
| I didnt know I did that                                        |      35 |
| I dont know                                                    |      25 |

In [8]:
##### for phone
query = ("select * from survey2 where permission='PHONE'")
cursor.execute(query)

allowAll={"I want to use a specific feature that requires this permission":0,
          "I think the app wont work otherwise":0,
          "I trust the app developer":0,
          "Because the app is popular":0,
          "I wont be able to grant this permission later":0,
          "I have nothing to hide":0,
          "I wanted the permission screen to go away":0,
          "Nothing bad will happen":0,
          "I didnt know I did that":0,
          "I dont know":0,
          "The app developer already has this information about me":0,
          "Other":0}
disAll={"I do not use the specific feature associated with the permission":0,
        "I think the app shouldnt need this permission":0,
        "I expect the app will still work without this permission":0,
        "I consider the permission to be very sensitive":0,
        "I dont trust the developer enough to provide this information":0,
        "I can always grant it afterwards if I change my mind":0,
        "I wanted the permission screen to go away":0,
        "I think something bad might happen if I provide this permission":0,
        "I didnt know I did that":0,
        "I dont know":0,
        "I dont use the app":0,
        "The app did not ask for that permission/I did not disable the permission/I think thats the default setting":0,
        "Other":0}

for row in cursor:
    reasonY=[]
    reasonN=[]
    reasonY1 = row[42].split(',')
    reasonY.append(reasonY1)
    reasonY2 = row[43].split(',')
    reasonY.append(reasonY2)
    reasonY3 = row[44].split(',')
    reasonY.append(reasonY3)
    reasonY4 = row[45].split(',')
    reasonY.append(reasonY4)
    reasonY5 = row[46].split(',')
    reasonY.append(reasonY5)
    
    reasonN1 = row[53].split(',')
    reasonN.append(reasonN1)
    reasonN2 = row[55].split(',')
    reasonN.append(reasonN2)
    reasonN3 = row[57].split(',')
    reasonN.append(reasonN3)
    reasonN4 = row[59].split(',')
    reasonN.append(reasonN4)
    reasonN5 = row[61].split(',')
    reasonN.append(reasonN5)
    
    for x1 in reasonY:
        for x in x1:
            x = x.lstrip()
            x = x.rstrip()
            x=x.replace('\'','')
            x=x.replace('\\','')
            if x[0:5] == "Other": allowAll["Other"] += 1
            elif x=="NA": pass
            elif x == "I want to use a specific feature that requires this permission": allowAll[x] += 1
            elif x == "I think the app wont work otherwise": allowAll[x] += 1
            elif x == "I trust the app developer": allowAll[x] += 1
            elif x == "Because the app is popular": allowAll[x] += 1
            elif x=="I wont be able to grant this permission later":allowAll[x] += 1
            elif x== "I have nothing to hide":allowAll[x] += 1
            elif x== "I wanted the permission screen to go away":allowAll[x] += 1
            elif x== "Nothing bad will happen":allowAll[x] += 1
            elif x== "I didnt know I did that":allowAll[x] += 1
            elif x== "I dont know":allowAll[x] += 1
            elif x== "The app developer already has this information about me":allowAll[x] += 1        
        
    for x1 in reasonN:
        for x in x1:
             
            x = x.lstrip()
            x = x.rstrip()
            x=x.replace('\'','')
            x=x.replace('\\','')
            x=x.replace("thatâ€™s","thats")
            x=x.replace("donâ€™t","dont")
            if x[0:5] == "Other": disAll["Other"] += 1
            elif x=="NA": pass
            elif x=="I do not use the specific feature associated with the permission": disAll[x] += 1
            elif x=="I think the app shouldnt need this permission": disAll[x] += 1
            elif x=="I expect the app will still work without this permission": disAll[x] += 1
            elif x=="I consider the permission to be very sensitive": disAll[x] += 1
            elif x=="I dont trust the developer enough to provide this information": disAll[x] += 1
            elif x=="I can always grant it afterwards if I change my mind": disAll[x] += 1
            elif x=="I wanted the permission screen to go away": disAll[x]+= 1
            elif x=="I think something bad might happen if I provide this permission": disAll[x] += 1
            elif x=="I didnt know I did that": disAll[x] += 1
            elif x=="I dont know": disAll[x] += 1
            elif x=="I dont use the app": disAll[x] += 1
            elif x=="The app did not ask for that permission/I did not disable the permission/I think thats the default setting": disAll[x] += 1

allAllow = []
for key in allowAll:
    temp = []
    temp.append(key)
    temp.append(allowAll[key])
    allAllow.append(temp)
allDis = []
for key in disAll:
    temp = []
    #make this spesific key shorter for display purposes 
    if key== "The app did not ask for that permission/I did not disable the permission/I think thats the default setting":
        temp.append("The app didn't ask for it/I didn't disable it/I think thats the default setting"+'')    
    else:
         temp.append(key+'')  
    temp.append(disAll[key])
    allDis.append(temp)



allowM = pandas.DataFrame(allAllow,columns=['Reason:PHONE:Allow','Count'])
disallowM = pandas.DataFrame(allDis,columns=['Reason:PHONE:Deny','Count'])
print(tabulate(allowM, showindex=False, headers=allowM.columns,tablefmt='psql'))
print(tabulate(disallowM, showindex=False, headers=disallowM.columns,tablefmt='psql'))
#pandas.set_option('max_colwidth',200)

+----------------------------------------------------------------+---------+
| Reason:PHONE:Allow                                             |   Count |
|----------------------------------------------------------------+---------|
| I want to use a specific feature that requires this permission |      41 |
| I think the app wont work otherwise                            |      23 |
| I trust the app developer                                      |      27 |
| Because the app is popular                                     |       7 |
| I wont be able to grant this permission later                  |       0 |
| I have nothing to hide                                         |      17 |
| I wanted the permission screen to go away                      |      10 |
| Nothing bad will happen                                        |      10 |
| I didnt know I did that                                        |      43 |
| I dont know                                                    |      12 |

In [9]:
##### for storage
query = ("select * from survey2 where permission='STORAGE'")
cursor.execute(query)

allowAll={"I want to use a specific feature that requires this permission":0,
          "I think the app wont work otherwise":0,
          "I trust the app developer":0,
          "Because the app is popular":0,
          "I wont be able to grant this permission later":0,
          "I have nothing to hide":0,
          "I wanted the permission screen to go away":0,
          "Nothing bad will happen":0,
          "I didnt know I did that":0,
          "I dont know":0,
          "The app developer already has this information about me":0,
          "Other":0}
disAll={"I do not use the specific feature associated with the permission":0,
        "I think the app shouldnt need this permission":0,
        "I expect the app will still work without this permission":0,
        "I consider the permission to be very sensitive":0,
        "I dont trust the developer enough to provide this information":0,
        "I can always grant it afterwards if I change my mind":0,
        "I wanted the permission screen to go away":0,
        "I think something bad might happen if I provide this permission":0,
        "I didnt know I did that":0,
        "I dont know":0,
        "I dont use the app":0,
        "The app did not ask for that permission/I did not disable the permission/I think thats the default setting":0,
        "Other":0}

for row in cursor:
    reasonY=[]
    reasonN=[]
    reasonY1 = row[42].split(',')
    reasonY.append(reasonY1)
    reasonY2 = row[43].split(',')
    reasonY.append(reasonY2)
    reasonY3 = row[44].split(',')
    reasonY.append(reasonY3)
    reasonY4 = row[45].split(',')
    reasonY.append(reasonY4)
    reasonY5 = row[46].split(',')
    reasonY.append(reasonY5)
    
    reasonN1 = row[53].split(',')
    reasonN.append(reasonN1)
    reasonN2 = row[55].split(',')
    reasonN.append(reasonN2)
    reasonN3 = row[57].split(',')
    reasonN.append(reasonN3)
    reasonN4 = row[59].split(',')
    reasonN.append(reasonN4)
    reasonN5 = row[61].split(',')
    reasonN.append(reasonN5)
    
    for x1 in reasonY:
        for x in x1:
            x = x.lstrip()
            x = x.rstrip()
            x=x.replace('\'','')
            x=x.replace('\\','')
            if x[0:5] == "Other": allowAll["Other"] += 1
            elif x=="NA": pass
            elif x == "I want to use a specific feature that requires this permission": allowAll[x] += 1
            elif x == "I think the app wont work otherwise": allowAll[x] += 1
            elif x == "I trust the app developer": allowAll[x] += 1
            elif x == "Because the app is popular": allowAll[x] += 1
            elif x=="I wont be able to grant this permission later":allowAll[x] += 1
            elif x== "I have nothing to hide":allowAll[x] += 1
            elif x== "I wanted the permission screen to go away":allowAll[x] += 1
            elif x== "Nothing bad will happen":allowAll[x] += 1
            elif x== "I didnt know I did that":allowAll[x] += 1
            elif x== "I dont know":allowAll[x] += 1
            elif x== "The app developer already has this information about me":allowAll[x] += 1        
        
    for x1 in reasonN:
        for x in x1:
             
            x = x.lstrip()
            x = x.rstrip()
            x=x.replace('\'','')
            x=x.replace('\\','')
            x=x.replace("thatâ€™s","thats")
            x=x.replace("donâ€™t","dont")
            if x[0:5] == "Other": disAll["Other"] += 1
            elif x=="NA": pass
            elif x=="I do not use the specific feature associated with the permission": disAll[x] += 1
            elif x=="I think the app shouldnt need this permission": disAll[x] += 1
            elif x=="I expect the app will still work without this permission": disAll[x] += 1
            elif x=="I consider the permission to be very sensitive": disAll[x] += 1
            elif x=="I dont trust the developer enough to provide this information": disAll[x] += 1
            elif x=="I can always grant it afterwards if I change my mind": disAll[x] += 1
            elif x=="I wanted the permission screen to go away": disAll[x]+= 1
            elif x=="I think something bad might happen if I provide this permission": disAll[x] += 1
            elif x=="I didnt know I did that": disAll[x] += 1
            elif x=="I dont know": disAll[x] += 1
            elif x=="I dont use the app": disAll[x] += 1
            elif x=="The app did not ask for that permission/I did not disable the permission/I think thats the default setting": disAll[x] += 1

allAllow = []
for key in allowAll:
    temp = []
    temp.append(key)
    temp.append(allowAll[key])
    allAllow.append(temp)
allDis = []
for key in disAll:
    temp = []
    #make this spesific key shorter for display purposes 
    if key== "The app did not ask for that permission/I did not disable the permission/I think thats the default setting":
        temp.append("The app didn't ask for it/I didn't disable it/I think thats the default setting"+'')    
    else:
         temp.append(key+'')  
    temp.append(disAll[key])
    allDis.append(temp)



allowM = pandas.DataFrame(allAllow,columns=['Reason:STORAGE:Allow','Count'])
disallowM = pandas.DataFrame(allDis,columns=['Reason:STORAGE:Deny','Count'])
print(tabulate(allowM, showindex=False, headers=allowM.columns,tablefmt='psql'))
print(tabulate(disallowM, showindex=False, headers=disallowM.columns,tablefmt='psql'))
#pandas.set_option('max_colwidth',200)

+----------------------------------------------------------------+---------+
| Reason:STORAGE:Allow                                           |   Count |
|----------------------------------------------------------------+---------|
| I want to use a specific feature that requires this permission |      64 |
| I think the app wont work otherwise                            |      59 |
| I trust the app developer                                      |      43 |
| Because the app is popular                                     |      11 |
| I wont be able to grant this permission later                  |       1 |
| I have nothing to hide                                         |      12 |
| I wanted the permission screen to go away                      |      15 |
| Nothing bad will happen                                        |      15 |
| I didnt know I did that                                        |      39 |
| I dont know                                                    |      34 |

In [10]:
cursor.close()
cnx.close()